# Final Coursera Project - Melbourne Restaurant Analysis

### The following notebook contains my final project and report.

#### Introduction/Business Problem

I am thinking of opeining an Italian restaurant in Melbourne but am unsure where to open it. It would be best to be outside of suburbs with a lot of existing Italian restaurants but also not too far away that people would not think to look for an Italian restaurant in that location. My business problem is to find a suburb adjacent to suburbs with a strong Italian restaurant presence.

#### Data Requirements

This report will analyse the ditribution of restaurant types in Melbourne suburbs using foursquare data (venues/explore). I will find a list of Melbourne suburbs online and scrape the data. I will then use geopy.geocoders to find the latitude and longitude. I will find the number of Italian restaurants by suburb, and also the most common restaurant type by suburb and visualize the results using folium maps.

I have not completed the week 5 project yet so please disregard the code below for now. :-)

Firstly, import the required libraries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Part 1: Reading in Melbourne surbub data and finding lat long with geocode.

In [3]:
#Defining Url path
url = "https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs"

In [4]:
#Use pandas to read html
url_df = pd.read_html(url, header=0)

In [5]:
#Pull the table of suburbs from the html
df = url_df[0]
df.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

   Location[citation needed]  Distance[3][citation needed]  \
0                        NaN                           NaN   
1                        NaN                           NaN   
2                        NaN                           NaN   
3                        NaN                           NaN   
4                        NaN                           NaN   

  Area[citation needed] Population[citation needed]  \
0               0.9 km2                    1,793[4]   
1                   NaN                    3,152[4]   
2                15 km2                       28653   
3               1.9 km2                        3873   
4                   NaN                         NaN   

   Population density[citation needed]  Date established[citation needed]  
0                                  NaN                                NaN  
1                                  NaN                                NaN  
2                                  NaN                                NaN  
3                                  NaN                                NaN  
4                                  NaN                                NaN

In [6]:
#Format the address for geocode search
df['Address'] = df['Suburb'] + ', VIC ' + df['Postcode'].apply(str) + ", Australia"

In [7]:
#Creat a geolocator
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [8]:
#Create function to pull lat and long from location
def geoConvLat(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.latitude
    else: return 0
def geoConvLong(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.longitude
    else: return 0

In [9]:
#Find lat
#lats = []
#for i in df['Address']:
#    lats.append(geoConvLat(i))

In [10]:
#Find long
#longs = []
#for i in df['Address']:
#    longs.append(geoConvLong(i))

In [11]:
#Create lat and long columns in the df
#lats = pd.Series(lats)
#longs = pd.Series(longs)
#df['Latitude'] = lats
#df['Longitude'] = longs

In [12]:
#save the df (lat and long took a long time)
#df.to_csv('data/suburbs_and_coords.csv')

### Load Suburbs with Coordinates

In [15]:
#Read in suburbs and coords. Checkpoint
df = pd.read_csv('data/suburbs_and_coords.csv')
df.shape

(549, 13)

In [16]:
#drop irrelevant fields
drop_columns = ['Population[citation needed]', 'Area[citation needed]', 'Population density[citation needed]', 
                'Unnamed: 0', 'Location[citation needed]', 'Distance[3][citation needed]', 'Date established[citation needed]',
               'Address']
df = df.drop(drop_columns, axis=1)

In [17]:
#check the df
df.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

    Latitude   Longitude  
0 -37.753107  145.038478  
1 -37.706370  145.121648  
2 -37.697306  145.066254  
3 -37.765144  145.063331  
4 -37.713387  145.147880

In [18]:
#Plot them on a folium map
import folium

# create map using latitude and longitude values
map_melbourne = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# add markers to map
for lat, lng, suburb, lga in zip(df['Latitude'], df['Longitude'], df['Suburb'], df['Local government area']):
    label = '{}, {}'.format(suburb, lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

###  Part 2: Foursquare Section

In [20]:
#Define my details for a Foursquare query

CLIENT_ID = 'LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML' # your Foursquare ID
CLIENT_SECRET = 'AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM' # your Foursquare Secret
VERSION = '20190701' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML
CLIENT_SECRET:AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM


In [21]:
#Define the details of the query itself

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
lat, long = (-37.9136, 144.9631)
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML&client_secret=AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM&v=20190701&ll=-37.9136,144.9631&radius=1000&limit=100'

In [22]:
#Using function from Coursera labs to get nearby venues for each suburbs

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#Getting nearby venues for each suburb

melbourne_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [26]:
melbourne_venues.groupby('Suburb')['Suburb'].count()
print("Done")

Done


In [27]:
print('There are {} uniques categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 218 uniques categories.


### Part 3: Data Manipulation

In [28]:
# one hot encoding
onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Suburb'] = melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0  Bellfield               0                  0             0         0   
1  Bellfield               0                  0             0         0   
2  Bellfield               0                  0             0         0   
3  Bellfield               0                  0             0         0   
4  Bellfield               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Arts & Crafts Store  \
0       0                       0            0                    0   
1       0                       0            0                    0   
2       0                       0            0                    0   
3       0                       0            0                    0   
4       0                       0            0                    0   

   Asian Restaurant  ...  Tunnel  Turkish Restaurant  \
0                 0  ...       0                   0   
1                 0  ...       0                   0   
2                 0  ...       0                   0   
3                 0  ...       0                   0   
4                 0  ...       0                   0   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              0            0                      0   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  0  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          0              0                  0  

[5 rows x 219 columns]

In [29]:
onehot.shape

(8155, 219)

In [30]:
grouped = onehot.groupby('Suburb').sum().reset_index()
grouped.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0    Abbotsford               0                  0             0         0   
1    Aberfeldie               0                  1             0         0   
2       Aintree               0                  0             0         0   
3  Airport West               0                  0             0         0   
4     Albanvale               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Arts & Crafts Store  \
0       0                       0            0                    0   
1       0                       0            0                    0   
2       0                       0            0                    0   
3       0                       0            0                    0   
4       0                       0            0                    0   

   Asian Restaurant  ...  Tunnel  Turkish Restaurant  \
0                 1  ...       0                   1   
1                 0  ...       0                   1   
2                 0  ...       0                   0   
3                 0  ...       0                   0   
4                 0  ...       0                   1   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              1            0                      1   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  1  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          1              0                  0  

[5 rows x 219 columns]

In [31]:
grouped.shape

(317, 219)

In [32]:
grouped.head()

Suburb  Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  \
0    Abbotsford               0                  0             0         0   
1    Aberfeldie               0                  1             0         0   
2       Aintree               0                  0             0         0   
3  Airport West               0                  0             0         0   
4     Albanvale               0                  0             0         0   

   Arcade  Argentinian Restaurant  Art Gallery  Arts & Crafts Store  \
0       0                       0            0                    0   
1       0                       0            0                    0   
2       0                       0            0                    0   
3       0                       0            0                    0   
4       0                       0            0                    0   

   Asian Restaurant  ...  Tunnel  Turkish Restaurant  \
0                 1  ...       0                   1   
1                 0  ...       0                   1   
2                 0  ...       0                   0   
3                 0  ...       0                   0   
4                 0  ...       0                   1   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                              1            0                      1   
1                              0            0                      0   
2                              0            0                      0   
3                              0            0                      0   
4                              0            0                      0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0           0         0          0              0                  1  
1           0         0          0              0                  0  
2           0         0          0              0                  0  
3           0         0          0              0                  0  
4           0         0          1              0                  0  

[5 rows x 219 columns]

In [33]:
merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(grouped.set_index('Suburb'), on='Suburb')

merged.tail()

Suburb  Postcode  Local government area   Latitude   Longitude  \
544   Woori Yallock      3139  Shire of Yarra Ranges -37.779333  145.530126   
545      Yarra Glen      3775  Shire of Yarra Ranges -37.657348  145.374396   
546  Yarra Junction      3797  Shire of Yarra Ranges -37.782169  145.615026   
547       Yellingbo      3139  Shire of Yarra Ranges -37.813513  145.508205   
548          Yering      3770  Shire of Yarra Ranges -37.688713  145.374657   

     Adult Boutique  Afghan Restaurant  Antique Shop  Aquarium  Arcade  ...  \
544             0.0                0.0           0.0       0.0     0.0  ...   
545             0.0                0.0           0.0       0.0     0.0  ...   
546             0.0                0.0           0.0       0.0     0.0  ...   
547             0.0                0.0           0.0       0.0     0.0  ...   
548             0.0                0.0           0.0       0.0     0.0  ...   

     Tunnel  Turkish Restaurant  Vegetarian / Vegan Restaurant  Video Store  \
544     0.0                 0.0                            0.0          1.0   
545     0.0                 0.0                            0.0          0.0   
546     0.0                 0.0                            0.0          0.0   
547     0.0                 0.0                            1.0          0.0   
548     0.0                 0.0                            0.0          0.0   

     Vietnamese Restaurant  Whisky Bar  Wine Bar  Wine Shop  Women's Store  \
544                    1.0         0.0       0.0        0.0            0.0   
545                    0.0         0.0       0.0        0.0            0.0   
546                    0.0         0.0       0.0        0.0            0.0   
547                    0.0         1.0       1.0        0.0            0.0   
548                    0.0         0.0       0.0        0.0            0.0   

     Yunnan Restaurant  
544                0.0  
545                0.0  
546                0.0  
547                0.0  
548                0.0  

[5 rows x 223 columns]

In [34]:
merged = merged.fillna(0)

In [35]:
merged.to_csv('data/mergedData.csv', index=None)

# Load Suburbs Merged with Venues

In [36]:
merged= pd.read_csv('data/mergedData.csv')
keepSuburbs = pd.read_csv('data/keepSuburbs.csv')
keepVenue = pd.read_csv('data/venues.csv')

In [37]:
keepList = []
for i in keepSuburbs['Suburb']:
    keepList.append(i)

keepList[:2]

['Beaumaris', 'Black Rock']

In [38]:
merged.head()

Suburb  Postcode                              Local government area  \
0   Bellfield      3081                                    City of Banyule   
1  Briar Hill      3088                                    City of Banyule   
2    Bundoora      3083  City of Banyule; City of Darebin; City of Whit...   
3   Eaglemont      3084                                    City of Banyule   
4      Eltham      3095                City of Banyule; Shire of Nillumbik   

    Latitude   Longitude  Adult Boutique  Afghan Restaurant  Antique Shop  \
0 -37.753107  145.038478             0.0                0.0           0.0   
1 -37.706370  145.121648             0.0                0.0           0.0   
2 -37.697306  145.066254             0.0                0.0           0.0   
3 -37.765144  145.063331             0.0                1.0           0.0   
4 -37.713387  145.147880             0.0                0.0           0.0   

   Aquarium  Arcade  ...  Tunnel  Turkish Restaurant  \
0       0.0     0.0  ...     0.0                 0.0   
1       0.0     0.0  ...     0.0                 0.0   
2       0.0     0.0  ...     0.0                 0.0   
3       0.0     0.0  ...     0.0                 1.0   
4       0.0     0.0  ...     0.0                 0.0   

   Vegetarian / Vegan Restaurant  Video Store  Vietnamese Restaurant  \
0                            0.0          0.0                    0.0   
1                            0.0          0.0                    0.0   
2                            0.0          0.0                    0.0   
3                            0.0          0.0                    1.0   
4                            0.0          0.0                    0.0   

   Whisky Bar  Wine Bar  Wine Shop  Women's Store  Yunnan Restaurant  
0         0.0       0.0        0.0            0.0                0.0  
1         0.0       0.0        0.0            0.0                0.0  
2         0.0       0.0        0.0            0.0                0.0  
3         0.0       0.0        0.0            0.0                0.0  
4         0.0       0.0        0.0            0.0                0.0  

[5 rows x 223 columns]

In [39]:
keepVenues = ['Suburb', 'Postcode', 'Local government area', 'Latitude', 'Longitude']
for i in keepVenue['Venue_Type']:
    keepVenues.append(i)

keepVenues[:2]

['Suburb', 'Postcode']

In [40]:
merged = merged[keepVenues]
merged = merged[merged['Suburb'].isin(keepList)]
merged = merged[merged['Suburb'].isin(keepList)]

In [41]:
merged['Italian Restaurant'].unique()

array([ 0.,  1.,  3.,  2., 15.,  5.,  6.,  9.])

In [42]:
merged.shape

(78, 49)

In [43]:
house_prices = pd.read_excel('data/melbourneSalesPrices.xlsx', index=None)

In [44]:
house_prices.shape

(48427, 2)

In [45]:
houses_grouped = house_prices.groupby('Suburb').mean().reset_index()
houses_grouped.sort_values(by='Price')[:5]

Suburb          Price
108           Darley  380000.000000
204        Kurunjang  381785.714286
224     Melton South  396267.500000
90          Cockatoo  397000.000000
240  Mount Dandenong  405000.000000

In [46]:
joined_df = merged.join(houses_grouped.set_index('Suburb'), on='Suburb', how='inner')
joined_df.head()

Suburb  Postcode              Local government area   Latitude  \
21      Beaumaris      3193                    City of Bayside -37.982212   
22     Black Rock      3193                    City of Bayside -37.970519   
23       Brighton      3186                    City of Bayside -37.905608   
24  Brighton East      3187                    City of Bayside -37.917173   
25     Cheltenham      3192  City of Bayside; City of Kingston -37.963418   

     Longitude  Middle Eastern Restaurant  Mediterranean Restaurant  \
21  145.038910                        0.0                       0.0   
22  145.022952                        0.0                       0.0   
23  145.002751                        0.0                       0.0   
24  145.016366                        0.0                       0.0   
25  145.061567                        0.0                       0.0   

    Indian Restaurant  Japanese Restaurant  Portuguese Restaurant  ...  \
21                0.0                  0.0                    0.0  ...   
22                0.0                  0.0                    0.0  ...   
23                0.0                  0.0                    0.0  ...   
24                0.0                  0.0                    0.0  ...   
25                0.0                  0.0                    0.0  ...   

    Modern European Restaurant  Yunnan Restaurant  Persian Restaurant  \
21                         0.0                0.0                 0.0   
22                         0.0                0.0                 0.0   
23                         0.0                0.0                 0.0   
24                         0.0                0.0                 0.0   
25                         0.0                0.0                 0.0   

    Egyptian Restaurant  Theme Restaurant  Gluten-free Restaurant  \
21                  0.0               0.0                     0.0   
22                  0.0               0.0                     0.0   
23                  0.0               0.0                     0.0   
24                  0.0               0.0                     0.0   
25                  0.0               0.0                     0.0   

    Cantonese Restaurant  South American Restaurant  Shanghai Restaurant  \
21                   0.0                        0.0                  0.0   
22                   0.0                        0.0                  0.0   
23                   0.0                        0.0                  0.0   
24                   0.0                        0.0                  0.0   
25                   0.0                        0.0                  0.0   

           Price  
21  1.514327e+06  
22  1.541980e+06  
23  2.027742e+06  
24  1.686790e+06  
25  9.691274e+05  

[5 rows x 50 columns]

### Part 4: K Means Clustering

In [47]:
# set number of clusters
kclusters = 5

merged_clustering = joined_df.drop(['Suburb', 'Postcode', 'Local government area', 'Latitude', 'Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42, n_init=100).fit(merged_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 0, 3, 1, 1, 1, 3, 1, 1, 1, 3, 3, 0, 3, 3, 0, 0, 3, 2, 2, 3,
       3, 2, 3, 2, 1, 4, 1, 1, 2, 4, 2, 4, 2, 1, 1, 2, 2, 4, 2, 4, 4, 4,
       2, 2, 4, 1, 1, 0, 4, 1, 0, 4, 4, 2, 3, 3, 0, 3, 1, 3, 1, 1, 1, 2,
       4, 1, 2, 2, 3, 1])

In [48]:
joined_df.insert(column ='Cluster', value = kmeans.labels_, loc = 1)
joined_df.head()

Suburb  Cluster  Postcode              Local government area  \
21      Beaumaris        3      3193                    City of Bayside   
22     Black Rock        3      3193                    City of Bayside   
23       Brighton        0      3186                    City of Bayside   
24  Brighton East        3      3187                    City of Bayside   
25     Cheltenham        1      3192  City of Bayside; City of Kingston   

     Latitude   Longitude  Middle Eastern Restaurant  \
21 -37.982212  145.038910                        0.0   
22 -37.970519  145.022952                        0.0   
23 -37.905608  145.002751                        0.0   
24 -37.917173  145.016366                        0.0   
25 -37.963418  145.061567                        0.0   

    Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  ...  \
21                       0.0                0.0                  0.0  ...   
22                       0.0                0.0                  0.0  ...   
23                       0.0                0.0                  0.0  ...   
24                       0.0                0.0                  0.0  ...   
25                       0.0                0.0                  0.0  ...   

    Modern European Restaurant  Yunnan Restaurant  Persian Restaurant  \
21                         0.0                0.0                 0.0   
22                         0.0                0.0                 0.0   
23                         0.0                0.0                 0.0   
24                         0.0                0.0                 0.0   
25                         0.0                0.0                 0.0   

    Egyptian Restaurant  Theme Restaurant  Gluten-free Restaurant  \
21                  0.0               0.0                     0.0   
22                  0.0               0.0                     0.0   
23                  0.0               0.0                     0.0   
24                  0.0               0.0                     0.0   
25                  0.0               0.0                     0.0   

    Cantonese Restaurant  South American Restaurant  Shanghai Restaurant  \
21                   0.0                        0.0                  0.0   
22                   0.0                        0.0                  0.0   
23                   0.0                        0.0                  0.0   
24                   0.0                        0.0                  0.0   
25                   0.0                        0.0                  0.0   

           Price  
21  1.514327e+06  
22  1.541980e+06  
23  2.027742e+06  
24  1.686790e+06  
25  9.691274e+05  

[5 rows x 51 columns]

In [49]:
# create map
map_clusters = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
import math
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, n, p in zip(joined_df['Latitude'], joined_df['Longitude'], joined_df['Suburb'], 
                                     joined_df['Cluster'], joined_df['Italian Restaurant'], joined_df['Price']):
    label = folium.Popup(str(poi) + ", Cluster: " + str(cluster) + ", Price: " + str(p//1) + 
                         ", Italian Restuarants: " + str(int(n)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=1
    ).add_to(map_clusters)
    
       
map_clusters

### Part 5: Visualising the Results

In [50]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [51]:
joined_df.loc[joined_df['Cluster'] == 0].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count     7.00      7.00      7.00       7.00                       7.00   
mean      0.00   3153.43    -37.84     145.03                       0.00   
std       0.00     45.67      0.03       0.05                       0.00   
min       0.00   3103.00    -37.91     144.96                       0.00   
25%       0.00   3114.50    -37.85     144.98                       0.00   
50%       0.00   3144.00    -37.85     145.04                       0.00   
75%       0.00   3196.00    -37.82     145.07                       0.00   
max       0.00   3206.00    -37.81     145.08                       0.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                      7.00               7.00                 7.00   
mean                       0.00               0.14                 0.86   
std                        0.00               0.38                 1.07   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 2.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                   7.00                  7.00                 7.00   
mean                    0.14                  0.57                 0.00   
std                     0.38                  0.98                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  1.00                 0.00   
max                     1.00                  2.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count                7.00                7.00                   7.00   
mean                 0.86                0.00                   0.00   
std                  1.46                0.00                   0.00   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  1.50                0.00                   0.00   
max                  3.00                0.00                   0.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count             7.00                7.00               7.00   
mean              0.29                0.00               0.00   
std               0.49                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.50                0.00               0.00   
max               1.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                         7.00                 7.00              7.00   
mean                          0.00                 0.29              0.86   
std                           0.00                 0.76              1.21   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              1.50   
max                           0.00                 2.00              3.00   

       Kebab Restaurant  Italian Restaura

In [52]:
joined_df.loc[joined_df['Cluster'] == 1].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    20.00     20.00     20.00      20.00                      20.00   
mean      1.00   3154.35    -37.88     145.02                       0.10   
std       0.00     49.22      0.05       0.04                       0.31   
min       1.00   3003.00    -37.96     144.92                       0.00   
25%       1.00   3136.00    -37.91     145.00                       0.00   
50%       1.00   3173.00    -37.88     145.01                       0.00   
75%       1.00   3185.75    -37.84     145.04                       0.00   
max       1.00   3204.00    -37.80     145.07                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     20.00              20.00                20.00   
mean                       0.00               0.15                 0.40   
std                        0.00               0.37                 0.99   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.00   
max                        0.00               1.00                 3.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  20.00                 20.00                20.00   
mean                    0.10                  0.10                 0.00   
std                     0.31                  0.45                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  2.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               20.00               20.00                  20.00   
mean                 0.25                0.00                   0.10   
std                  0.72                0.00                   0.31   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  3.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            20.00               20.00              20.00   
mean              0.40                0.05               0.00   
std               0.94                0.22               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.25                0.00               0.00   
max               4.00                1.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        20.00                20.00             20.00   
mean                          0.00                 0.05              0.20   
std                           0.00                 0.22              0.62   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              0.00   
max                           0.00                 1.00              2.00   

       Kebab Restaurant  Italian Restaura

In [53]:
joined_df.loc[joined_df['Cluster'] == 2].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    16.00     16.00     16.00      16.00                      16.00   
mean      2.00   3118.81    -37.83     145.00                       0.06   
std       0.00     68.68      0.05       0.05                       0.25   
min       2.00   3002.00    -37.92     144.92                       0.00   
25%       2.00   3062.25    -37.85     144.97                       0.00   
50%       2.00   3122.50    -37.82     144.99                       0.00   
75%       2.00   3189.75    -37.80     145.03                       0.00   
max       2.00   3207.00    -37.78     145.11                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     16.00              16.00                16.00   
mean                       0.00               0.19                 0.50   
std                        0.00               0.40                 0.89   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.50   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  16.00                 16.00                16.00   
mean                    0.06                  0.19                 0.00   
std                     0.25                  0.40                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  1.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               16.00               16.00                  16.00   
mean                 0.38                0.00                   0.12   
std                  0.81                0.00                   0.34   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.25                0.00                   0.00   
max                  3.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            16.00               16.00              16.00   
mean              0.69                0.00               0.00   
std               0.79                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.50                0.00               0.00   
75%               1.00                0.00               0.00   
max               2.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        16.00                16.00             16.00   
mean                          0.00                 0.19              0.62   
std                           0.00                 0.54              0.81   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.50   
75%                           0.00                 0.00              1.00   
max                           0.00                 2.00              3.00   

       Kebab Restaurant  Italian Restaura

In [54]:
joined_df.loc[joined_df['Cluster'] == 3].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    17.00     17.00     17.00      17.00                      17.00   
mean      3.00   3143.00    -37.87     145.04                       0.00   
std       0.00     39.37      0.06       0.03                       0.00   
min       3.00   3054.00    -37.98     144.97                       0.00   
25%       3.00   3124.00    -37.92     145.02                       0.00   
50%       3.00   3144.00    -37.86     145.04                       0.00   
75%       3.00   3187.00    -37.83     145.07                       0.00   
max       3.00   3193.00    -37.78     145.10                       0.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     17.00              17.00                17.00   
mean                       0.00               0.12                 0.12   
std                        0.00               0.33                 0.49   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.00                 0.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  17.00                 17.00                17.00   
mean                    0.00                  0.06                 0.00   
std                     0.00                  0.24                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     0.00                  1.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               17.00               17.00                  17.00   
mean                 0.00                0.00                   0.06   
std                  0.00                0.00                   0.24   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  0.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            17.00               17.00              17.00   
mean              0.47                0.00               0.00   
std               1.01                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               1.00                0.00               0.00   
max               4.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        17.00                17.00             17.00   
mean                          0.00                 0.00              0.18   
std                           0.00                 0.00              0.53   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              0.00   
max                           0.00                 0.00              2.00   

       Kebab Restaurant  Italian Restaura

In [55]:
joined_df.loc[joined_df['Cluster'] == 4].describe()

Cluster  Postcode  Latitude  Longitude  Middle Eastern Restaurant  \
count    12.00     12.00     12.00      12.00                      12.00   
mean      4.00   3102.58    -37.84     144.98                       0.08   
std       0.00     75.63      0.04       0.04                       0.29   
min       4.00   3006.00    -37.89     144.92                       0.00   
25%       4.00   3031.00    -37.88     144.94                       0.00   
50%       4.00   3114.00    -37.84     144.98                       0.00   
75%       4.00   3167.75    -37.80     145.00                       0.00   
max       4.00   3185.00    -37.79     145.06                       1.00   

       Mediterranean Restaurant  Indian Restaurant  Japanese Restaurant  \
count                     12.00              12.00                12.00   
mean                       0.00               0.25                 0.50   
std                        0.00               0.45                 0.80   
min                        0.00               0.00                 0.00   
25%                        0.00               0.00                 0.00   
50%                        0.00               0.00                 0.00   
75%                        0.00               0.25                 1.00   
max                        0.00               1.00                 2.00   

       Portuguese Restaurant  Fast Food Restaurant  Lebanese Restaurant  \
count                  12.00                 12.00                12.00   
mean                    0.08                  0.00                 0.00   
std                     0.29                  0.00                 0.00   
min                     0.00                  0.00                 0.00   
25%                     0.00                  0.00                 0.00   
50%                     0.00                  0.00                 0.00   
75%                     0.00                  0.00                 0.00   
max                     1.00                  0.00                 0.00   

       Mexican Restaurant  Falafel Restaurant  Vietnamese Restaurant  \
count               12.00               12.00                  12.00   
mean                 0.00                0.00                   0.17   
std                  0.00                0.00                   0.39   
min                  0.00                0.00                   0.00   
25%                  0.00                0.00                   0.00   
50%                  0.00                0.00                   0.00   
75%                  0.00                0.00                   0.00   
max                  0.00                0.00                   1.00   

       Thai Restaurant  Turkish Restaurant  Afghan Restaurant  \
count            12.00               12.00              12.00   
mean              0.42                0.00               0.00   
std               0.79                0.00               0.00   
min               0.00                0.00               0.00   
25%               0.00                0.00               0.00   
50%               0.00                0.00               0.00   
75%               0.25                0.00               0.00   
max               2.00                0.00               0.00   

       Eastern European Restaurant  Dumpling Restaurant  Sushi Restaurant  \
count                        12.00                12.00             12.00   
mean                          0.00                 0.08              0.42   
std                           0.00                 0.29              0.51   
min                           0.00                 0.00              0.00   
25%                           0.00                 0.00              0.00   
50%                           0.00                 0.00              0.00   
75%                           0.00                 0.00              1.00   
max                           0.00                 1.00              1.00   

       Kebab Restaurant  Italian Restaura

In [56]:
cluster_ave_price = pd.DataFrame([joined_df.loc[joined_df['Cluster'] == 0]['Price'].mean(), joined_df.loc[joined_df['Cluster'] == 1]['Price'].mean(),
joined_df.loc[joined_df['Cluster'] == 2]['Price'].mean(), joined_df.loc[joined_df['Cluster'] == 3]['Price'].mean(),
joined_df.loc[joined_df['Cluster'] == 4]['Price'].mean()],columns=['Average Price'])
cluster_ave_price

Average Price
0     2104943.87
1     1067327.42
2     1332251.53
3     1647958.21
4      815137.76

In [57]:
cluster_ave_italian = pd.DataFrame([joined_df.loc[joined_df['Cluster'] == 0]['Italian Restaurant'].mean(), 
joined_df.loc[joined_df['Cluster'] == 1]['Italian Restaurant'].mean(),
joined_df.loc[joined_df['Cluster'] == 2]['Italian Restaurant'].mean(), 
joined_df.loc[joined_df['Cluster'] == 3]['Italian Restaurant'].mean(),
joined_df.loc[joined_df['Cluster'] == 4]['Italian Restaurant'].mean()], columns=['Ave Italian Restaurants'])
cluster_ave_italian

Ave Italian Restaurants
0                     0.86
1                     0.80
2                     1.50
3                     0.94
4                     1.42

In [58]:
joined_df.loc[joined_df['Cluster'] == 1][['Suburb', 'Italian Restaurant', 'Price']]

Suburb  Italian Restaurant      Price
25        Cheltenham                0.00  969127.39
26        Gardenvale                0.00 1095312.50
169       Gardenvale                0.00 1095312.50
28      Hampton East                0.00 1096513.04
29           Highett                0.00 1083245.06
30         Moorabbin                0.00 1009232.99
162   Bentleigh East                0.00 1132489.32
164   Caulfield East                0.00 1035750.00
165  Caulfield North                0.00  958154.49
172      Murrumbeena                0.00 1061576.90
173           Ormond                0.00 1096426.63
295      South Yarra                1.00 1078120.07
296   West Melbourne                1.00 1041723.08
429           Elwood                0.00 1011153.37
439          Prahran                0.00 1149590.55
441          Windsor                0.00 1051843.75
478       Abbotsford                9.00 1040200.70
479          Burnley                0.00 1191875.62
482         Cremorne                0.00 1076294.12
486         Richmond                5.00 1072606.23

In [59]:
best_suburb = joined_df.loc[joined_df['Cluster'] == 1][['Suburb', 'Cluster', 'Latitude', 'Longitude', 
                                                        'Postcode', 'Local government area', 'Italian Restaurant', 'Price']]
best_suburb.loc[best_suburb['Suburb'] == 'Burnley']

Suburb  Cluster  Latitude  Longitude  Postcode Local government area  \
479  Burnley        1    -37.83     145.02      3121         City of Yarra   

     Italian Restaurant      Price  
479                0.00 1191875.62

In [60]:
# create map using latitude and longitude values
map_burnley = folium.Map(location=(-37.83, 145.02), zoom_start=13)

label = '{}, {}'.format('Burnley', 'City of Yarra')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [-37.83, 145.0175],
    radius=40,
    popup=label,
    color='blue',
    fill=True,
    fill_opacity=0.2,
    parse_html=False).add_to(map_burnley)  

map_burnley